# AI Video Processing Pipeline - Google Colab Setup

This notebook sets up and tests the complete video processing pipeline:
1. **CodeFormer** - Face restoration
2. **LivePortrait** - Portrait animation
3. **Real-ESRGAN** - Video upscaling
4. **FILM** - Frame interpolation

**Note**: Enable GPU in Runtime → Change runtime type → GPU (T4)

## Step 1: Install Dependencies

In [ ]:
# Install system dependencies
!apt-get update
!apt-get install -y ffmpeg

# Install Python packages
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install tensorflow==2.8.0
!pip install numpy==1.26.4
!pip install protobuf==3.20.3
!pip install opencv-python pillow scipy scikit-image imageio
!pip install fastapi uvicorn
!pip install basicsr facexlib gfpgan
!pip install tensorflow-addons==0.18.0
!pip install tensorflow-datasets==4.4.0
!pip install apache-beam>=2.43.0,<2.63.0
!pip install mediapy absl-py gin-config parameterized natsort gdown tqdm
!pip install transformers onnxruntime-gpu
!pip install gradio tyro pyyaml

print("✓ Dependencies installed!")

## Step 2: Clone Repositories

In [ ]:
import os
os.chdir('/content')

# Clone repositories
!git clone https://github.com/sczhou/CodeFormer.git
!git clone https://github.com/KlingTeam/LivePortrait.git
!git clone https://github.com/xinntao/Real-ESRGAN.git
!git clone https://github.com/google-research/frame-interpolation.git

print("✓ Repositories cloned!")

## Step 3: Fix Compatibility Issues

In [ ]:
# Fix basicsr torchvision import (Real-ESRGAN)
import re

degradations_path = '/content/Real-ESRGAN/realesrgan/data/degradations.py'
if os.path.exists(degradations_path):
    with open(degradations_path, 'r') as f:
        content = f.read()
    content = content.replace(
        'from torchvision.transforms.functional_tensor import rgb_to_grayscale',
        'from torchvision.transforms.functional import rgb_to_grayscale'
    )
    with open(degradations_path, 'w') as f:
        f.write(content)
    print("✓ Fixed Real-ESRGAN torchvision import")

# Create version.py files
version_files = [
    ('/content/Real-ESRGAN/realesrgan/version.py', '0.3.0'),
    ('/content/CodeFormer/basicsr/version.py', '1.3.2')
]

for filepath, version in version_files:
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, 'w') as f:
        f.write(f"""# GENERATED VERSION FILE
# TIME: Generated manually
__version__ = '{version}'
__gitsha__ = 'unknown'
version_info = {tuple(map(int, version.split('.')))}
""")
    print(f"✓ Created {filepath}")

# Fix frame-interpolation tf.data.AUTOTUNE
data_lib_path = '/content/frame-interpolation/training/data_lib.py'
if os.path.exists(data_lib_path):
    with open(data_lib_path, 'r') as f:
        content = f.read()
    content = content.replace('tf.data.experimental.AUTOTUNE', 'tf.data.AUTOTUNE')
    with open(data_lib_path, 'w') as f:
        f.write(content)
    print("✓ Fixed frame-interpolation tf.data.AUTOTUNE")

# Fix Real-ESRGAN video inference CPU handling
video_inference_path = '/content/Real-ESRGAN/inference_realesrgan_video.py'
if os.path.exists(video_inference_path):
    with open(video_inference_path, 'r') as f:
        content = f.read()
    
    # Fix num_process for CPU
    content = re.sub(
        r'num_process = num_gpus \* args\.num_process_per_gpu',
        r'num_process = max(1, num_gpus * args.num_process_per_gpu)  # Ensure at least 1 process for CPU mode',
        content
    )
    
    # Fix CUDA synchronization
    content = content.replace(
        '        torch.cuda.synchronize(device)',
        '        # Only synchronize if using CUDA device\n        if device is not None and device.type == \'cuda\':\n            torch.cuda.synchronize(device)'
    )
    
    with open(video_inference_path, 'w') as f:
        f.write(content)
    print("✓ Fixed Real-ESRGAN video inference")

print("\n✓ All compatibility fixes applied!")

## Step 4: Download Pretrained Weights

In [ ]:
import gdown
from pathlib import Path

# Create directories
os.makedirs('/content/pretrained_models', exist_ok=True)
os.makedirs('/content/CodeFormer/weights/CodeFormer', exist_ok=True)
os.makedirs('/content/Real-ESRGAN/weights', exist_ok=True)

print("Downloading model weights...")
print("This may take a while...")

# CodeFormer weights
print("\n[1/4] Downloading CodeFormer weights...")
codeformer_url = 'https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/codeformer.pth'
!wget -q {codeformer_url} -O /content/CodeFormer/weights/CodeFormer/codeformer.pth
print("✓ CodeFormer weights downloaded")

# Real-ESRGAN weights
print("\n[2/4] Downloading Real-ESRGAN weights...")
realesrgan_url = 'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth'
!wget -q {realesrgan_url} -O /content/Real-ESRGAN/weights/RealESRGAN_x4plus.pth
print("✓ Real-ESRGAN weights downloaded")

# LivePortrait weights (using HuggingFace)
print("\n[3/4] Downloading LivePortrait weights from HuggingFace...")
!pip install -q huggingface_hub
from huggingface_hub import snapshot_download
snapshot_download(
    repo_id="KlingTeam/LivePortrait",
    local_dir="/content/LivePortrait/pretrained_weights",
    ignore_patterns=["*.git*", "README.md", "docs"]
)
print("✓ LivePortrait weights downloaded")

# FILM weights (frame-interpolation)
print("\n[4/4] Downloading FILM weights from Google Drive...")
print("Note: You may need to download manually from:")
print("https://drive.google.com/drive/folders/1q8110-qp225asX3DQvZnfLfJPkCHmDpy?usp=sharing")
print("Or use gdown to download the folder...")

# Try using gdown for FILM weights
film_folder_id = '1q8110-qp225asX3DQvZnfLfJPkCHmDpy'
try:
    !gdown --folder https://drive.google.com/drive/folders/{film_folder_id} -O /content/pretrained_models --remaining-ok
    print("✓ FILM weights downloaded")
except:
    print("⚠ FILM weights download failed - you may need to download manually")

print("\n✓ Weight download complete!")

## Step 5: Test Individual Models

### Test 1: Real-ESRGAN (Image Upscaling)

In [ ]:
import sys
sys.path.insert(0, '/content/Real-ESRGAN')

from inference_realesrgan import main
import argparse

# Create test image directory
os.makedirs('/content/Real-ESRGAN/inputs', exist_ok=True)

# Upload a test image or use a sample
print("Upload a test image to /content/Real-ESRGAN/inputs/ or use the sample below")

# Test with sample (if you have one)
# args = argparse.Namespace(
#     input='/content/Real-ESRGAN/inputs/test.jpg',
#     model_name='RealESRGAN_x4plus',
#     output='/content/Real-ESRGAN/results',
#     outscale=4,
#     suffix='out',
#     tile=0,
#     tile_pad=10,
#     pre_pad=0,
#     face_enhance=False,
#     fp32=False,
#     alpha_upsampler='realesrgan',
#     ext='auto'
# )
# main(args)

print("Ready to test Real-ESRGAN!")

### Test 2: CodeFormer (Face Restoration)

In [ ]:
import sys
sys.path.insert(0, '/content/CodeFormer')

os.makedirs('/content/CodeFormer/inputs', exist_ok=True)

print("Ready to test CodeFormer!")
print("Upload test images to /content/CodeFormer/inputs/")
print("Then run:")
print("!cd /content/CodeFormer && python inference_codeformer.py -w 0.7 --input_path inputs/")

### Test 3: Frame Interpolation (FILM)

In [ ]:
import sys
sys.path.insert(0, '/content/frame-interpolation')

os.makedirs('/content/frame-interpolation/photos', exist_ok=True)

print("Ready to test FILM!")
print("Upload two test images to /content/frame-interpolation/photos/")
print("Then run:")
print("!cd /content/frame-interpolation && python3 -m eval.interpolator_test \\")
print("  --frame1 photos/one.png \\")
print("  --frame2 photos/two.png \\")
print("  --model_path /content/pretrained_models/film_net/Style/saved_model \\")
print("  --output_frame photos/output.png")

### Test 4: LivePortrait (Portrait Animation)

In [ ]:
import sys
sys.path.insert(0, '/content/LivePortrait')

print("Ready to test LivePortrait!")
print("Upload source image and driving video, then run:")
print("!cd /content/LivePortrait && python inference.py -s path/to/source.jpg -d path/to/driving.mp4")

## Step 6: Test Full Pipeline (Optional)

If you have the pipeline code, you can test the complete workflow:

In [ ]:
# If you upload your pipeline code, you can test it here
# from models.pipeline import VideoPipeline
# 
# pipeline = VideoPipeline(device='cuda')
# output = pipeline.process_video(
#     input_video_path='input.mp4',
#     audio_path='audio.wav',
#     output_path='output.mp4'
# )

print("Pipeline ready for testing!")

## Notes

1. **GPU Required**: Enable GPU in Colab (Runtime → Change runtime type → GPU)
2. **Storage**: Colab provides ~80GB disk space, but model weights are large (~5-10GB total)
3. **Session Timeout**: Colab free tier sessions timeout after ~12 hours of inactivity
4. **Download Results**: Use Colab's file browser or download programmatically

## Quick Test Commands

After setup, test each model:

```bash
# Real-ESRGAN
cd /content/Real-ESRGAN
python inference_realesrgan.py -n RealESRGAN_x4plus -i inputs/test.jpg -o results

# CodeFormer
cd /content/CodeFormer
python inference_codeformer.py -w 0.7 --input_path inputs/

# FILM
cd /content/frame-interpolation
python3 -m eval.interpolator_test --frame1 photos/one.png --frame2 photos/two.png --model_path /content/pretrained_models/film_net/Style/saved_model --output_frame photos/output.png

# LivePortrait
cd /content/LivePortrait
python inference.py -s assets/examples/source/s6.jpg -d assets/examples/driving/d0.mp4
```